In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import cv2


sys.path.append(os.path.abspath(".."))

In [1]:
%cd ../Application_programming_with_OpenCV_Streamlit

[WinError 2] The system cannot find the file specified: '../Application_programming_with_OpenCV_Streamlit'
c:\Users\ACER FPT\Documents\Application_programming_with_OpenCV_Streamlit\notebooks


C:\Users\ACER FPT\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [10]:
DATASET_DIR  = ".\Application_programming_with_OpenCV_Streamlit\data\semantic_keypoint_detection\synthetic_shapes_datasets"

DATA_TYPES  = [
    os.path.join(DATASET_DIR, "draw_checkerboard"),
    os.path.join(DATASET_DIR, "draw_cube"),
    os.path.join(DATASET_DIR, "draw_ellipses"),
    os.path.join(DATASET_DIR, "draw_lines"),
    os.path.join(DATASET_DIR, "draw_multiple_polygons"),
    os.path.join(DATASET_DIR, "draw_polygon"),
    os.path.join(DATASET_DIR, "draw_star"),
    os.path.join(DATASET_DIR, "draw_stripes"),
]

In [ ]:
def calculate_precision_recall(ground_truth_points: np.ndarray, predicted_points: np.ndarray, distance_threshold=3):

    # Nếu không có điểm nào trong ground truth hoặc dự đoán, trả về (0, 0)
    if len(ground_truth_points) == 0 or len(predicted_points) == 0:
        return (0, 0)

    true_positive, false_negative, false_positive = 0, 0, 0

    # Đếm true positives và false positives
    for predicted_point in predicted_points:
        distances = np.linalg.norm(ground_truth_points - predicted_point, axis=1)
        if np.any(distances <= distance_threshold):
            true_positive += 1
        else:
            false_positive += 1

    # Đếm false negatives
    for ground_truth_point in ground_truth_points:
        distances = np.linalg.norm(predicted_points - ground_truth_point, axis=1)
        if not np.any(distances <= distance_threshold):
            false_negative += 1

    # Tính toán precision và recall
    if true_positive == 0:
        return (0, 0)

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    return (precision, recall)

In [14]:
# Tạo các đối tượng SIFT và ORB
sift_detector = cv2.SIFT_create()
orb_detector = cv2.ORB_create()

# Danh sách lưu kết quả Precision và Recall cho từng loại hình dạng
precision_recall_sift = []
precision_recall_orb = []

for shape_index in range(8):
    precision_recall_sift_shape = []
    precision_recall_orb_shape = []
    for image_index in range(500):
        # Tải ảnh và điểm thực tế từ thư mục tương ứng
        image_path = os.path.join(DATA_TYPES[shape_index], "images", f"{image_index}.png")
        ground_truth_path = os.path.join(DATA_TYPES[shape_index], "points", f"{image_index}.npy")
        
        image = cv2.imread(image_path)
        ground_truth = np.load(ground_truth_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Phát hiện điểm đặc trưng bằng SIFT và ORB
        detected_sift = sift_detector.detect(gray_image, None)
        detected_orb = orb_detector.detect(gray_image, None)

        # Chuyển đổi điểm phát hiện thành mảng numpy
        pred_sift = np.array([[kp.pt[1], kp.pt[0]] for kp in detected_sift])
        pred_orb = np.array([[kp.pt[1], kp.pt[0]] for kp in detected_orb])

        # Tính Precision và Recall
        precision_recall_sift_shape.append(calculate_precision_recall(ground_truth, pred_sift))
        precision_recall_orb_shape.append(calculate_precision_recall(ground_truth, pred_orb))

    precision_recall_sift.append(precision_recall_sift_shape)
    precision_recall_orb.append(precision_recall_orb_shape)


In [15]:
results_dir = ".\Application_programming_with_OpenCV_Streamlit\data\semantic_keypoint_detection"

# Lưu kết quả Precision và Recall cho SIFT và ORB vào file .npy
np.save(os.path.join(results_dir, "precision_recall_sift.npy"), precision_recall_sift)
np.save(os.path.join(results_dir, "precision_recall_orb.npy"), precision_recall_orb)

# In kích thước của các mảng kết quả để kiểm tra
print("Kích thước của precision_recall_sift:", np.shape(precision_recall_sift))
print("Kích thước của precision_recall_orb:", np.shape(precision_recall_orb))


Kích thước của precision_recall_sift: (8, 500, 2)
Kích thước của precision_recall_orb: (8, 500, 2)
